In [ ]:
for surgery in ['hip', 'knee'] :
    for yr in ['1', '2', '3', '0.5', '0.25'] :
        dataset = surgery + '_' + yr + 'yr'
        %run "Discovery Experiments"/run_models.py --dataset {dataset} --data_dir=. --output_dir='outputs/' \
        --exp_types lr lda vanilla attention --structured --display --bsize 8 --n_iters=20

In [ ]:
for surgery in ['hip', 'knee'] :
    for yr in ['1', '2', '3', '0.5', '0.25'] :
        dataset = surgery + '_' + yr
        %run "Discovery Experiments"/run_models.py --dataset {dataset} --data_dir=. --output_dir='outputs/' \
        --exp_types vanilla attention --display --bsize 8 --n_iters=20

In [1]:
from dataloaders import dataloaders

In [2]:
import argparse
parser = argparse.ArgumentParser(description='Run experiments on a dataset')
parser.add_argument("--data_dir", type=str, required=True)

args, extras = parser.parse_known_args(['--data_dir=.'])
args.extras = extras

dataset = dataloaders['both_1yr'](args)

INFO - 2019-04-13 15:20:39,856 - Reading Structured data ...
INFO - 2019-04-13 15:20:40,635 - Reading Notes ...
INFO - 2019-04-13 15:20:42,867 - Stratifying ...


In [11]:
train_data = dataset.filter_data_length(dataset.get_data('train', structured=False), 90)
dev_data = dataset.filter_data_length(dataset.get_data('dev', structured=False), 90)

Pos Percentage [0.04390465 0.44930597 0.05099578 0.55069403]


INFO - 2019-04-13 15:21:56,564 - Maximum Sentence Length 585530.000000 , 90 percentile length 19689.000000 ... 
INFO - 2019-04-13 15:21:58,877 - Truncated all ...
INFO - 2019-04-13 15:21:58,884 - Idxs removed []...
INFO - 2019-04-13 15:21:59,206 - Maximum Sentence Length 119600.000000 , 90 percentile length 20149.000000 ... 


Pos Percentage [0.04519774 0.4519774  0.06638418 0.5480226 ]


INFO - 2019-04-13 15:21:59,569 - Truncated all ...
INFO - 2019-04-13 15:21:59,571 - Idxs removed []...


In [4]:
from Experiments.modifiable_config_exp import LSTM

In [8]:
config = LSTM(dataset, False, None)

In [9]:
from models.Vanilla import ClassificationTrainer
trainer = ClassificationTrainer(config)

INFO - 2019-04-13 15:21:18,887 - instantiating class <class 'models.Model.Model'> from params {'type': 'seq_classifier', 'embedder': {'type': 'token_embedder', 'vocab_size': 26447, 'embed_size': 200, 'embedding_file': './preprocess/MultiTaskSurgery/1_yrs/embedding_matrix.npy'}, 'decoder': {'num_layers': 1, 'hidden_dims': [128], 'activations': ['tanh']}, 'predictor': {'type': 'multitask', 'n_tasks': 2, 'task_decoder': {'num_layers': 1, 'hidden_dims': [1], 'activations': ['linear'], 'input_dim': 128}}, 'structured': {'use_structured': False}, 'encoder': {'type': 'lstm', 'hidden_size': 128}} and extras {}
INFO - 2019-04-13 15:21:18,888 - type = seq_classifier
INFO - 2019-04-13 15:21:18,889 - instantiating class <class 'models.modules.Embedder.Embedder'> from params {'type': 'token_embedder', 'vocab_size': 26447, 'embed_size': 200, 'embedding_file': './preprocess/MultiTaskSurgery/1_yrs/embedding_matrix.npy'} and extras {}
INFO - 2019-04-13 15:21:18,889 - embedder.type = token_embedder
INFO

Setting Embedding


In [12]:
trainer.train(train_data)

TypeError: unhashable type: 'numpy.ndarray'